In [1]:
import pandas as pd

# Data: https://dpi.wi.gov/wisedash/download-files/type?field_wisedash_upload_type_value=All

In [2]:
year = ['17', '18', '19', '21']

In [3]:
def enrollment_clean(x, year):
    data = pd.read_csv(x).rename(columns = {'DISTRICT_CODE':'districtcode', 'SCHOOL_CODE':'schoolcode'})
    data = data[((data['GROUP_BY_VALUE'].isin(['Econ Disadv', 'Black', 'Hispanic', 'White'])) & 
                (data['schoolcode'].notna()))].loc[:, ['districtcode', 'schoolcode', 'GROUP_BY_VALUE', 
                'PERCENT_OF_GROUP']].pivot(index = ['districtcode', 'schoolcode'], columns = 'GROUP_BY_VALUE',
                                          values = 'PERCENT_OF_GROUP').reset_index().fillna(0)
    data = data[data['Econ Disadv'] != '*']
    data = data.rename(columns = {'Econ Disadv':'lowincome', 'Black':'black', 'Hispanic':'hispanic',
                          'White':'white'})
    data.loc[:, ['districtcode', 'schoolcode']] = data.loc[:, ['districtcode', 
                                                               'schoolcode']].astype('int').astype('str')
    data.loc[:, ['lowincome', 'black', 'hispanic', 'white']] = round(data.loc[:, 
    ['lowincome', 'black', 'hispanic', 'white']].astype('float'), 2)
    data['mergecode'] = data['districtcode'] + '-' + data['schoolcode']
    data['year'] = year
    data.set_index('year').to_csv(f'enrollment_clean_{year}.csv')

In [4]:
for i in year:
    enrollment_clean(f'enrollment{i}.csv', i)

In [5]:
data21 = set(list(pd.read_csv('enrollment_clean_21.csv').mergecode))
data19 = set(list(pd.read_csv('enrollment_clean_19.csv').mergecode))
data18 = set(list(pd.read_csv('enrollment_clean_18.csv').mergecode))
data17 = set(list(pd.read_csv('enrollment_clean_17.csv').mergecode))

mutual_code = list(data21.intersection(data19).intersection(data18).intersection(data17))

In [6]:
all_data_enroll = pd.DataFrame()
for i in year:
    data = pd.read_csv(f'enrollment_clean_{i}.csv')
    data = data.loc[data['mergecode'].isin(mutual_code)]
    all_data_enroll = pd.concat([all_data_enroll, data])

In [7]:
def dropout_clean(x, year):
    data = pd.read_csv(x).rename(columns = {'SCHOOL_CODE':'schoolcode', 'DISTRICT_CODE':'districtcode',
                                           'DROPOUT_RATE':'droprate', 'STUDENT_COUNT':'totalenroll'})
    data = data[((data['GROUP_BY_VALUE'] == 'All Students') & (data['schoolcode'].notna()) & 
                (data['GRADE_GROUP'] == 'High School'))].loc[:, ['districtcode', 'schoolcode', 
                                                                 'droprate', 'totalenroll']]
    data.loc[:, ['districtcode', 'schoolcode']] = data.loc[:, ['districtcode', 
                                                               'schoolcode']].astype('int').astype('str')
    data['mergecode'] = data['districtcode'] + '-' + data['schoolcode']
    data['year'] = year
    data.set_index('year').to_csv(f'dropout_clean_{year}.csv')

In [8]:
for i in year:
    dropout_clean(f'drop{i}.csv', i)

In [9]:
data21 = set(list(pd.read_csv('dropout_clean_21.csv').mergecode))
data19 = set(list(pd.read_csv('dropout_clean_19.csv').mergecode))
data18 = set(list(pd.read_csv('dropout_clean_18.csv').mergecode))
data17 = set(list(pd.read_csv('dropout_clean_17.csv').mergecode))

mutual_code = list(data21.intersection(data19).intersection(data18).intersection(data17))

In [10]:
all_data_drop = pd.DataFrame()
for i in year:
    data = pd.read_csv(f'dropout_clean_{i}.csv')
    data = data.loc[data['mergecode'].isin(mutual_code)]
    all_data_drop = pd.concat([all_data_drop, data])

In [11]:
drop = all_data_drop.loc[:, ['year', 'mergecode', 'totalenroll', 'droprate']].merge(
all_data_enroll,on = ['year', 'mergecode'])

In [12]:
def test_clean(x, year):

    data = pd.read_csv(x).rename(columns = {'SCHOOL_CODE':'schoolcode', 'DISTRICT_CODE':'districtcode'})
    data = data[((data['TEST_RESULT'].isin(['Proficient', 'Advanced'])) & (data['schoolcode'].notna()) & 
                 (data['TEST_SUBJECT'].isin(['ELA', 'Mathematics'])) & (data['TEST_GROUP'] == 'ACT') 
                 & (data['GROUP_BY'] == 'All Students'))].loc[:, ['districtcode', 'schoolcode',
                 'TEST_SUBJECT', 'STUDENT_COUNT', 'GROUP_COUNT']]
    totaltest = data.loc[:, ['districtcode', 'schoolcode', 'GROUP_COUNT']].drop_duplicates().rename(
        columns = {'GROUP_COUNT':'totaltest'})
    totaltest['totaltest'] = totaltest['totaltest'].astype('int')
    data['STUDENT_COUNT'] = data['STUDENT_COUNT'].astype('int')
    data = data.groupby(['districtcode', 'schoolcode', 'TEST_SUBJECT']).sum().loc[
    :, ['STUDENT_COUNT']].reset_index()
    data = data.merge(totaltest, on = ['districtcode', 'schoolcode'])
    data['passrate'] = round((data['STUDENT_COUNT'] / data['totaltest']) * 100, 2)
    data = data.loc[:, ['districtcode', 'schoolcode', 'TEST_SUBJECT', 'passrate']].pivot(
    index = ['districtcode', 'schoolcode'], columns = 'TEST_SUBJECT', values = 'passrate').reset_index()
    data = data.merge(totaltest, on = ['districtcode', 'schoolcode']).rename(columns = {'ELA':'elapass',
    'Mathematics':'mathpass'})
    data.loc[:, ['districtcode', 'schoolcode']] = data.loc[:, ['districtcode', 
                                                               'schoolcode']].astype('int').astype('str')
    data['mergecode'] = data['districtcode'] + '-' + data['schoolcode']
    data['year'] = year
    data.set_index('year').to_csv(f'act_clean_{year}.csv')
    

In [13]:
for i in year:
    test_clean(f'act{i}.csv', i)

In [14]:
data21 = set(list(pd.read_csv('act_clean_21.csv').mergecode))
data19 = set(list(pd.read_csv('act_clean_19.csv').mergecode))
data18 = set(list(pd.read_csv('act_clean_18.csv').mergecode))
data17 = set(list(pd.read_csv('act_clean_17.csv').mergecode))

mutual_code = list(data21.intersection(data19).intersection(data18).intersection(data17))

In [15]:
all_data_act = pd.DataFrame()
for i in year:
    data = pd.read_csv(f'act_clean_{i}.csv')
    data = data.loc[data['mergecode'].isin(mutual_code)]
    all_data_act = pd.concat([all_data_act, data]).fillna(0)

In [16]:
test = all_data_act.loc[:, ['year', 'mergecode', 'totaltest', 'mathpass', 'elapass']].merge(
all_data_enroll,on = ['year', 'mergecode']).merge(all_data_drop.loc[:, ['year', 'mergecode', 'totalenroll']],
                                                 on = ['year', 'mergecode'])

In [17]:
all_data = all_data_act.loc[:, ['year', 'mergecode', 'totaltest', 'mathpass', 'elapass']].merge(
drop, on = ['year', 'mergecode'])

In [18]:
# clean learning mode data
mode = pd.read_csv('learning_mode1.csv').rename(columns = {
    'StateAssignedSchoolID':'schoolcode', 'StateAssignedDistrictID':'districtcode'
})

mode.loc[:, ['schoolcode', 'districtcode']] = mode.loc[:, 
            ['schoolcode', 'districtcode']].astype('int').astype('str')

mode['mergecode'] = mode['districtcode'] + '-' + mode['schoolcode']

# remove closed period
mode = mode[mode['LearningModel']!='Closed']

# compute the charter status of schools
charter = mode.loc[:, ['mergecode', 'Charter']].drop_duplicates().rename(
    columns = {'Charter':'charter'})
charter_map = {'No':0, 'Yes':1}
charter['charter'] = charter['charter'].map(charter_map)

mode = mode.groupby(['mergecode', 'NCESSchoolID'])['LearningModel'].value_counts(
    normalize = True).to_frame().rename(columns = {'LearningModel':'normalized'}).reset_index()

mode = mode.pivot(columns = 'LearningModel', values = 'normalized', 
index = ['mergecode', 'NCESSchoolID']).reset_index().fillna(0).loc[:, ['mergecode', 'NCESSchoolID',
                                                                'Hybrid', 'Virtual', 'In-person']].rename(
columns = {'Hybrid':'hybridper', 'Virtual':'virtualper', 'In-person':'inpersonper'})

mode['schoolmode'] = 1 * mode['virtualper'] + 0.5 * mode['hybridper']

mode.loc[:, ['hybridper', 'virtualper', 'inpersonper', 'schoolmode']] = round(
    mode.loc[:, ['hybridper', 'virtualper', 'inpersonper', 'schoolmode']], 2)

mode['NCESSchoolID'] = mode['NCESSchoolID'].astype('str')

In [19]:
all_mode = mode.merge(all_data, on = ['mergecode'])
all_mode.loc[all_mode['year']!=21, ['hybridper', 'virtualper', 'schoolmode']] = 0
all_mode.loc[all_mode['year'] != 21, ['inpersonper']] = 1
all_mode['year_21'] = 1
all_mode.loc[all_mode['year'] != 21, ['year_21']] = 0
all_mode.loc[:, ['districtcode', 'year']] = all_mode.loc[:, ['districtcode', 'year']].astype('str')
all_mode['state'] = 'wisconsin'
all_mode['districtcode'] = all_mode['districtcode'].astype('str') + all_mode['state']
all_mode['mergecode'] = all_mode['mergecode'] + all_mode['state']
all_mode = all_mode.drop(columns = 'schoolcode')
all_mode.set_index('year').to_csv('wisconsin_all.csv')